In [6]:
import pandas as pd
import numpy as np


In [14]:
df_rate=pd.read_csv("ratings.csv")
df_rate

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [15]:
df_rate["timestamp"]=pd.to_datetime(df_rate["timestamp"],unit='s')

In [16]:
df_tag=pd.read_csv("tags.csv")
df_tag.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [17]:
grouped_tag = df_tag.groupby(['userId', 'movieId'])['tag'].apply('|'.join).reset_index()
grouped_tag

,userId,movieId,tag
0,2,60756,funny|Highly quotable|will ferrell
1,2,89774,Boxing story|MMA|Tom Hardy
2,2,106782,drugs|Leonardo DiCaprio|Martin Scorsese
3,7,48516,way too long
4,18,431,Al Pacino|gangster|mafia
...,...,...,...
1770,606,6107,World War II
1771,606,7382,for katie
1772,606,7936,austere
1773,610,3265,gun fu|heroic bloodshed


In [18]:
# Merge df1 and df2 based on 'userId' and 'movieId' 
merged_df = pd.merge(df_rate[['userId', 'movieId', 'rating','timestamp']],
                     grouped_tag[['userId', 'movieId', 'tag']],
                     how='left',
                     on=['userId', 'movieId'])

# Display the merged DataFrame
merged_df


,userId,movieId,rating,timestamp,tag
0,1,1,4.0,2000-07-30 18:45:03,NaN
1,1,3,4.0,2000-07-30 18:20:47,NaN
2,1,6,4.0,2000-07-30 18:37:04,NaN
3,1,47,5.0,2000-07-30 19:03:35,NaN
4,1,50,5.0,2000-07-30 18:48:51,NaN
...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,Heroic Bloodshed
100833,610,168250,5.0,2017-05-08 19:50:47,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,NaN


In [7]:
df_title=pd.read_csv("movies.csv")
df_title

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [20]:
len(df_title["movieId"].sort_values().unique())

9742

In [21]:

# Merge df1 and df2 based on 'userId' and 'movieId'
df = pd.merge(merged_df, df_title, on=['movieId'])
df.sort_values(by=["timestamp"],inplace=True)
df.head()

# Display the merged DataFrame



,userId,movieId,rating,timestamp,tag,title,genres
20218,429,595,5.0,1996-03-29 18:36:55,NaN,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
31820,429,432,3.0,1996-03-29 18:36:55,NaN,City Slickers II: The Legend of Curly's Gold (...,Adventure|Comedy|Western
76180,429,227,3.0,1996-03-29 18:36:55,NaN,Drop Zone (1994),Action|Thriller
29805,429,225,4.0,1996-03-29 18:36:55,NaN,Disclosure (1994),Drama|Thriller
31903,429,434,4.0,1996-03-29 18:36:55,NaN,Cliffhanger (1993),Action|Adventure|Thriller


In [84]:
df_link=pd.read_csv('links.csv')
df_link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [99]:
from PyMovieDb import IMDB
import json
def add_tt(x):
    x=str(x)
    reminders=7-len(x)
    tconst= f"tt{reminders*'0'}{x}"
    imdb = IMDB()
    res = imdb.get_by_id(tconst)
    if res =='{"status": 404, "message": "No Result Found!", "result_count": 0, "results": []}': 
          print("error")
          return np.nan
    else:
          data= json.loads(res)
          print(data["poster"])
          return data["poster"]
     
     
  
        
    
    
        

In [102]:
add_tt(114709)

https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_.jpg


'https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_.jpg'

In [103]:
df_link["link"]=df_link["imdbId"].apply(add_tt)

https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_.jpg
https://m.media-amazon.com/images/M/MV5BZTk2ZmUwYmEtNTcwZS00YmMyLWFkYjMtNTRmZDA3YWExMjc2XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_.jpg
https://m.media-amazon.com/images/M/MV5BMjQxM2YyNjMtZjUxYy00OGYyLTg0MmQtNGE2YzNjYmUyZTY1XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_.jpg
https://m.media-amazon.com/images/M/MV5BYzcyMDY2YWQtYWJhYy00OGQ2LTk4NzktYWJkNDYwZWJmY2RjXkEyXkFqcGdeQXVyMTA0MjU0Ng@@._V1_.jpg
https://m.media-amazon.com/images/M/MV5BOTEyNzg5NjYtNDU4OS00MWYxLWJhMTItYWU4NTkyNDBmM2Y0XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_.jpg
error
https://m.media-amazon.com/images/M/MV5BYjQ5ZjQ0YzQtOGY3My00MWVhLTgzNWItOTYwMTE5N2ZiMDUyXkEyXkFqcGdeQXVyNjUwMzI2NzU@._V1_.jpg
error
https://m.media-amazon.com/images/M/MV5BYWEyODkwYTItZDI1ZC00ODhiLWFkNGMtMTU3NTk2ZGMxYjE3XkEyXkFqcGdeQXVyMjUzOTY1NTc@._V1_.jpg
error
https://m.media-amazon.com/images/M/MV5BNjhkMmU0M2YtZDUwYi00OWE0LWI5NTktODBjNDc1M2ZlMjI4XkEyXkFqcGde

KeyboardInterrupt: 

In [65]:
df_link

,movieId,imdbId,tmdbId
0,1,tt0114709,862.0
1,2,tt0113497,8844.0
2,3,tt0113228,15602.0
3,4,tt0114885,31357.0
4,5,tt0113041,11862.0
...,...,...,...
9737,193581,tt5476944,432131.0
9738,193583,tt5914996,445030.0
9739,193585,tt6397426,479308.0
9740,193587,tt8391976,483455.0


In [67]:

# Merge df1 and df2 based on 'userId' and 'movieId'
df2 = pd.merge(df, df_link[["movieId","imdbId"]], on=['movieId'])
df2.sort_values(by=["timestamp"],inplace=True)
df2.head()

# Display the merged DataFrame



,userId,movieId,rating,timestamp,tag,title,genres,imdbId
0,429,595,5.0,1996-03-29 18:36:55,NaN,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,tt0101414
223,429,225,4.0,1996-03-29 18:36:55,NaN,Disclosure (1994),Drama|Thriller,tt0109635
275,429,434,4.0,1996-03-29 18:36:55,NaN,Cliffhanger (1993),Action|Adventure|Thriller,tt0106582
376,429,222,4.0,1996-03-29 18:36:55,NaN,Circle of Friends (1995),Drama|Romance,tt0112679
392,429,22,4.0,1996-03-29 18:36:55,NaN,Copycat (1995),Crime|Drama|Horror|Mystery|Thriller,tt0112722


In [105]:
df2.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
tag          0
title        0
genres       0
imdbId       0
dtype: int64

In [104]:
df2["tag"].fillna("-",inplace=True)

In [69]:
len(df2["userId"].unique())

610

In [106]:
df2

,userId,movieId,rating,timestamp,tag,title,genres,imdbId
0,429,595,5.0,1996-03-29 18:36:55,-,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,tt0101414
223,429,225,4.0,1996-03-29 18:36:55,-,Disclosure (1994),Drama|Thriller,tt0109635
275,429,434,4.0,1996-03-29 18:36:55,-,Cliffhanger (1993),Action|Adventure|Thriller,tt0106582
376,429,222,4.0,1996-03-29 18:36:55,-,Circle of Friends (1995),Drama|Romance,tt0112679
392,429,22,4.0,1996-03-29 18:36:55,-,Copycat (1995),Crime|Drama|Horror|Mystery|Thriller,tt0112722
...,...,...,...,...,...,...,...,...
100724,514,187031,2.5,2018-09-23 03:55:27,-,Jurassic World: Fallen Kingdom (2018),Action|Adventure|Drama|Sci-Fi|Thriller,tt4881806
100654,514,187595,3.0,2018-09-23 03:55:46,-,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,tt3778644
63759,514,5247,2.5,2018-09-24 02:44:00,-,Smokey and the Bandit (1977),Action|Comedy,tt0076729
63688,514,5246,1.5,2018-09-24 02:44:19,-,Smokey and the Bandit II (1980),Action|Comedy,tt0081529


In [107]:
df2.to_csv("merged.csv",index=False)

In [12]:
df["userId"].sort_values().unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [38]:
from PyMovieDb import IMDB
import json
imdb = IMDB()
res = imdb.get_by_id("tt12593682")

In [41]:
d= json.loads(res)

d


In [43]:
d["poster"]

'https://m.media-amazon.com/images/M/MV5BMDU2ZmM2OTYtNzIxYy00NjM5LTliNGQtN2JmOWQzYTBmZWUzXkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_.jpg'

In [5]:
df_link1=pd.read_csv("df_link1.csv",index_col="Unnamed: 0")
df_link2=pd.read_csv("df_link2.csv",index_col="Unnamed: 0")
df_link3=pd.read_csv("df_link3.csv",index_col="Unnamed: 0")
df_link4=pd.read_csv("df_link4.csv",index_col="Unnamed: 0")
df_link5=pd.read_csv("df_link5.csv",index_col="Unnamed: 0")

In [6]:
frames = [df_link1, df_link2, df_link3,df_link4,df_link5]
result = pd.concat(frames)

In [7]:
result

,movieId,imdbId,tmdbId,poster
0,1,114709,862.0,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,113497,8844.0,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,113228,15602.0,https://m.media-amazon.com/images/M/MV5BMjQxM2...
3,4,114885,31357.0,https://m.media-amazon.com/images/M/MV5BYzcyMD...
4,5,113041,11862.0,https://m.media-amazon.com/images/M/MV5BOTEyNz...
...,...,...,...,...
9737,193581,5476944,432131.0,https://m.media-amazon.com/images/M/MV5BYjEyZT...
9738,193583,5914996,445030.0,https://m.media-amazon.com/images/M/MV5BMzkwY2...
9739,193585,6397426,479308.0,https://m.media-amazon.com/images/M/MV5BYzVlNW...
9740,193587,8391976,483455.0,https://m.media-amazon.com/images/M/MV5BYmNjNW...


In [9]:
result.to_csv("links_poster.csv",index=False)

In [22]:
df2 = pd.merge(df, result[["movieId","poster"]], on=['movieId'])
df2.sort_values(by=["timestamp"],inplace=True)
df2.head()


,userId,movieId,rating,timestamp,tag,title,genres,poster
0,429,595,5.0,1996-03-29 18:36:55,NaN,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,https://m.media-amazon.com/images/M/MV5BMzE5MD...
223,429,225,4.0,1996-03-29 18:36:55,NaN,Disclosure (1994),Drama|Thriller,https://m.media-amazon.com/images/M/MV5BNGQxNT...
275,429,434,4.0,1996-03-29 18:36:55,NaN,Cliffhanger (1993),Action|Adventure|Thriller,https://m.media-amazon.com/images/M/MV5BNDllNG...
376,429,222,4.0,1996-03-29 18:36:55,NaN,Circle of Friends (1995),Drama|Romance,https://icon-library.com/images/404-error-icon...
392,429,22,4.0,1996-03-29 18:36:55,NaN,Copycat (1995),Crime|Drama|Horror|Mystery|Thriller,https://icon-library.com/images/404-error-icon...


In [1]:
df2["tag"].fillna("-",inplace=True)

NameError: name 'df2' is not defined

In [25]:
df2.to_csv("merged.csv",index=False)

In [13]:
lst=df_title["title"].to_list()
lst

['Toy Story (1995)',
 'Jumanji (1995)',
 'Grumpier Old Men (1995)',
 'Waiting to Exhale (1995)',
 'Father of the Bride Part II (1995)',
 'Heat (1995)',
 'Sabrina (1995)',
 'Tom and Huck (1995)',
 'Sudden Death (1995)',
 'GoldenEye (1995)',
 'American President, The (1995)',
 'Dracula: Dead and Loving It (1995)',
 'Balto (1995)',
 'Nixon (1995)',
 'Cutthroat Island (1995)',
 'Casino (1995)',
 'Sense and Sensibility (1995)',
 'Four Rooms (1995)',
 'Ace Ventura: When Nature Calls (1995)',
 'Money Train (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Assassins (1995)',
 'Powder (1995)',
 'Leaving Las Vegas (1995)',
 'Othello (1995)',
 'Now and Then (1995)',
 'Persuasion (1995)',
 'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Dangerous Minds (1995)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Babe (1995)',
 'Dead Man Walking (1995)',
 'It Takes Two (1995)',
 'Clueless (1995)',
 'Cry, the Beloved Country

In [19]:
file_path = "user_list.txt"

# Convert the list to a string
user_list_str = ",\n".join(f'"{item}"' for item in lst)

# Open the file in write mode
with open(file_path, "w") as file:
    # Write the string representation of the list to the file
    file.write(user_list_str)